In [45]:
import pandas as pd
from geopy.geocoders import Nominatim

In [46]:
influence_df = pd.read_csv("data/influence.csv").drop("Unnamed: 6", axis = 1)

In [47]:
names_df = influence_df.applymap(lambda text : text.replace("http://webprotege.stanford.edu/","").replace("%C3%A8","è").strip(" ").strip('"'))

In [48]:
names_df

,"""previous_event""","""first_link_place""","""event""","""event_place""","""influenced_event""","""second_link_place"""
0,,,Purchase_Eisenlohr_Wheatstone,Karlsruhe,Meeting_Hipp_Eisenlohr,Karlsruhe
1,Purchase_Eisenlohr_Wheatstone,Karlsruhe,Meeting_Hipp_Eisenlohr,Karlsruhe,Creation_Hipp_Chronoscope,Neuchatel
2,Meeting_Hipp_Eisenlohr,Karlsruhe,Creation_Hipp_Chronoscope,Neuchatel,Borrow_Hirsch_Hipp,Neuchatel
3,Creation_Hipp_Chronoscope,Neuchatel,Borrow_Hirsch_Hipp,Neuchatel,Publication_Hirsch_Wundt,Neuchatel
4,Borrow_Hirsch_Hipp,Neuchatel,Publication_Hirsch_Wundt,Neuchatel,Letter_Wundt_Claparède,Leipzig
5,Borrow_Hirsch_Hipp,Neuchatel,Publication_Hirsch_Wundt,Leipzig,Letter_Wundt_Claparède,Leipzig
6,Publication_Hirsch_Wundt,Leipzig,Letter_Wundt_Claparède,Leipzig,,
7,Publication_Hirsch_Wundt,Leipzig,Letter_Wundt_Claparède,Genève,,


In [49]:
geolocator = Nominatim(user_agent="Hipp")

def get_coord(text):
    try:
        coord = geolocator.geocode(place)
    except: 
        try:
            coord = geolocator.geocode(place)
        except:
            return None
    
    return str(coord.latitude) + ", " + str(coord.longitude)

In [50]:
names_df = names_df.rename(columns=lambda x : x.strip(" ").strip('"'))
places = names_df[["first_link_place","event_place","second_link_place"]].stack().unique()

In [55]:
dict_coords = {}

for place in places:
    if len(place) > 0:
        print(place)
        dict_coords[place] = get_coord(place)

Karlsruhe
Neuchatel
Leipzig
Genève


In [56]:
dict_coords

{'Karlsruhe': '49.0068705, 8.4034195',
 'Neuchatel': '46.9895828, 6.9292641',
 'Leipzig': '51.3406321, 12.3747329',
 'Genève': '46.2017559, 6.1466014'}

In [66]:
coords_df = names_df[["first_link_place","event_place","second_link_place"]].applymap(lambda text : dict_coords.get(text))

In [68]:
labelled_coords_df =\
pd.merge(names_df[["previous_event","event","influenced_event"]], coords_df, left_index = True, right_index = True)

In [85]:
final_df = pd.concat(\
                     (labelled_coords_df[["previous_event","first_link_place","event","event_place"]].\
                      rename(columns = {"previous_event":"init_event","first_link_place":"init_place",\
                             "event":"target_event","event_place":"target_place"}),\
                     labelled_coords_df[["event","event_place","influenced_event","second_link_place"]].\
                     rename(columns = {"event":"init_event","event_place":"init_place",\
                             "influenced_event":"target_event","second_link_place":"target_place"})),\
                     axis = 0).dropna()

In [86]:
final_df

,init_event,init_place,target_event,target_place
1,Purchase_Eisenlohr_Wheatstone,"49.0068705, 8.4034195",Meeting_Hipp_Eisenlohr,"49.0068705, 8.4034195"
2,Meeting_Hipp_Eisenlohr,"49.0068705, 8.4034195",Creation_Hipp_Chronoscope,"46.9895828, 6.9292641"
3,Creation_Hipp_Chronoscope,"46.9895828, 6.9292641",Borrow_Hirsch_Hipp,"46.9895828, 6.9292641"
4,Borrow_Hirsch_Hipp,"46.9895828, 6.9292641",Publication_Hirsch_Wundt,"46.9895828, 6.9292641"
5,Borrow_Hirsch_Hipp,"46.9895828, 6.9292641",Publication_Hirsch_Wundt,"51.3406321, 12.3747329"
6,Publication_Hirsch_Wundt,"51.3406321, 12.3747329",Letter_Wundt_Claparède,"51.3406321, 12.3747329"
7,Publication_Hirsch_Wundt,"51.3406321, 12.3747329",Letter_Wundt_Claparède,"46.2017559, 6.1466014"
0,Purchase_Eisenlohr_Wheatstone,"49.0068705, 8.4034195",Meeting_Hipp_Eisenlohr,"49.0068705, 8.4034195"
1,Meeting_Hipp_Eisenlohr,"49.0068705, 8.4034195",Creation_Hipp_Chronoscope,"46.9895828, 6.9292641"
2,Creation_Hipp_Chronoscope,"46.9895828, 6.9292641",Borrow_Hirsch_Hipp,"46.9895828, 6.9292641"


In [87]:
final_df.to_csv("data/influence_processed.csv")